In [2]:
import sys
sys.path.append('..')
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sfp_nsdsyn import *
import warnings
import itertools
import matplotlib as mpl
import scipy

warnings.filterwarnings("ignore", category=UserWarning)
pd.options.mode.chained_assignment = None
%load_ext autoreload
%autoreload 2

output_dir='/Volumes/server/Projects/sfp_nsd/derivatives'
results_dir = os.path.join(output_dir, 'sfp_model', 'results_2D')
fig_dir = os.path.join(output_dir, 'figures')
lr = 0.0005
max_epoch = 30000
roi_list = ['V1','V2','V3']
vsc='pRFsize'
nsd_subj_list = [utils.sub_number_to_string(sn, 'nsdsyn') for sn in np.arange(1,9)]
broderick_sn_list = [1, 6, 7, 45, 46, 62, 64, 81, 95, 114, 115, 121]
broderick_subj_list = [utils.sub_number_to_string(i, 'broderick') for i in broderick_sn_list]
ARGS_2D = ['sub','lr','eph','roi']
roi_pal = [sns.color_palette('dark', 10)[:][k] for k in  [3,2,0]]
roi_pal.insert(0, (0.3,0.3,0.3))
stim_classes=['annulus', 'pinwheel', 'forward spiral','reverse spiral']

In [4]:
pt_file_list = []
for subj, roi in itertools.product(nsd_subj_list, roi_list):
    file_name = os.path.join(output_dir, 'sfp_model', 'results_2D', 'nsdsyn', 'corrected',
                             f'model-params_lr-{lr}_eph-{max_epoch}_sub-{subj}_roi-{roi}_vs-{vsc}.pt')
    pt_file_list.append(file_name)
nsd_params = model.load_all_models(pt_file_list, *ARGS_2D)

nsd_precision_s =  pd.read_csv(os.path.join(output_dir, 'dataframes', 'nsdsyn', 'precision', f'precision-s_dset-nsdsyn_vs-pRFsize.csv'))
nsd_df = pd.merge(nsd_params, nsd_precision_s[['sub','vroinames','precision']], on=['sub','vroinames'])

In [9]:
pt_file_list = []
for subj, roi in itertools.product(broderick_subj_list, ['V1']):
    file_name = os.path.join(output_dir,'sfp_model', 'results_2D', 'broderick','corrected', f'model-params_lr-{lr}_eph-{max_epoch}_sub-{subj}_roi-{roi}_vs-{vsc}.pt')
    pt_file_list.append(file_name)
broderick_params = model.load_all_models(pt_file_list, *ARGS_2D)

broderick_precision_s =  pd.read_csv(os.path.join(output_dir, 'dataframes', 'broderick', 'precision','corrected', f'precision-s_dset-broderick_vs-pRFsize.csv'))
broderick_df = pd.merge(broderick_params, broderick_precision_s[['sub','vroinames','precision']], on=['sub','vroinames'])

broderick_df['dset_type'] = 'Broderick et al. V1'
nsd_df['dset_type'] = nsd_df['vroinames'].apply(lambda x: f'NSD {x}')
final_params = pd.concat((broderick_df, nsd_df), axis=0)

In [31]:

PARAMS_2D = ['sigma', 'slope', 'intercept', 'p_1', 'p_2', 'p_3', 'p_4', 'A_1', 'A_2']
dset_types = ['Broderick et al. V1', 'NSD V1', 'NSD V2', 'NSD V3']
all_dset_df_mean = utils.calculate_weighted_mean(final_params, PARAMS_2D, 'precision', groupby=['dset_type'])
all_dset_df_68 = utils.calculate_confidence_intervals(final_params, PARAMS_2D, dset_types, ci_quantiles=[0.16, 0.84], n_boot=5000, seed=None)
all_dset_df_95 = utils.calculate_confidence_intervals(final_params, PARAMS_2D, dset_types, ci_quantiles=[0.025, 0.975], n_boot=5000, seed=None)


In [ ]:
# Combine all_dset_df_mean, all_dset_df_68, and all_dset_df_95
combined_df = all_dset_df_mean.copy()

# Iterate over each dataset type and parameter
for dset_type in dset_types:
    for param in PARAMS_2D:
        # Get mean, ci_68, and ci_95 for this param and dset_type
        mean_val = combined_df.loc[combined_df['dset_type'] == dset_type, param].values[0]
        ci_68_val = all_dset_df_68.loc[all_dset_df_68['dset_type'] == dset_type, f"{param}"].values[0]
        ci_95_val = all_dset_df_95.loc[all_dset_df_95['dset_type'] == dset_type, f"{param}"].values[0]
        
        # Format as requested: 'mean, [ci_68], [ci_95]'
        combined_df.loc[combined_df['dset_type'] == dset_type, param] = f"{mean_val}, {ci_68_val}, {ci_95_val}"


combined_df


,param,0,1,2,3
0,dset_type,Broderick et al. V1,NSD V1,NSD V2,NSD V3
1,sigma,"2.276, [2.147, 2.388], [2.068, 2.506]","2.199, [2.109, 2.293], [2.036, 2.379]","3.798, [3.403, 4.173], [3.222, 4.631]","4.461, [4.163, 4.827], [3.903, 5.202]"
2,slope,"0.115, [0.098, 0.132], [0.081, 0.146]","0.15, [0.136, 0.163], [0.121, 0.176]","0.171, [0.139, 0.208], [0.115, 0.249]","0.168, [0.136, 0.195], [0.1, 0.219]"
3,intercept,"0.372, [0.325, 0.421], [0.294, 0.484]","0.171, [0.137, 0.203], [0.099, 0.234]","0.254, [0.157, 0.329], [0.068, 0.393]","0.202, [0.127, 0.263], [0.058, 0.314]"
4,p_1,"0.071, [0.053, 0.089], [0.037, 0.11]","0.081, [0.049, 0.12], [0.012, 0.135]","0.162, [0.122, 0.201], [0.085, 0.233]","0.052, [0.009, 0.098], [-0.021, 0.151]"
5,p_2,"-0.028, [-0.034, -0.022], [-0.041, -0.018]","-0.022, [-0.035, -0.011], [-0.051, -0.002]","-0.103, [-0.131, -0.074], [-0.161, -0.046]","-0.047, [-0.076, -0.008], [-0.103, 0.011]"
6,p_3,"0.058, [0.035, 0.081], [0.012, 0.104]","-0.037, [-0.095, 0.018], [-0.149, 0.059]","-0.342, [-0.563, -0.128], [-0.796, -0.02]","-0.464, [-0.709, -0.239], [-0.918, -0.055]"
7,p_4,"-0.005, [-0.017, 0.007], [-0.027, 0.019]","-0.066, [-0.09, -0.045], [-0.113, -0.024]","0.039, [-0.042, 0.117], [-0.084, 0.203]","0.042, [-0.055, 0.133], [-0.135, 0.228]"
8,A_1,"0.045, [0.037, 0.053], [0.028, 0.061]","0.058, [0.046, 0.067], [0.033, 0.074]","0.046, [0.037, 0.054], [0.026, 0.06]","0.044, [0.031, 0.055], [0.021, 0.065]"
9,A_2,"-0.005, [-0.009, -0.001], [-0.013, 0.003]","-0.019, [-0.024, -0.015], [-0.028, -0.012]","-0.025, [-0.033, -0.016], [-0.042, -0.008]","-0.012, [-0.017, -0.007], [-0.021, -0.002]"


In [ ]:
save_path = '/Users/jh7685/Projects/spatial-frequency-preferences_NSDsyn/data/mean_parameter_estimates_with_ci.csv'
combined_df.to_csv(save_path, index=False)


2.101635967205492 2.278887254402971


In [ ]:
dset = ['NSD V1','NSD V2', 'NSD V3']
dset_df = final_params.query('dset_type in @dset')
print(dset_df['sub'].nunique())
bootstraps = utils.bootstrap_weighted_mean(dset_df, PARAMS_2D, 'precision', n_samples=8, groupby=['dset_type'])
bootstraps.query('dset_type == "NSD V1"')


8


,dset_type,sigma,slope,intercept,p_1,p_2,p_3,p_4,A_1,A_2
0,NSD V1,1.941282,0.182858,0.104924,0.139021,-0.030022,0.025469,-0.037078,0.057934,-0.012302
1,NSD V2,4.943823,0.148010,0.340602,0.138472,-0.042648,-0.771884,0.229674,0.068632,-0.009683
2,NSD V3,4.648829,0.106372,0.299997,0.065731,-0.081947,-0.018869,-0.232628,0.041831,-0.023357
3,NSD V1,2.365176,0.118251,0.130331,0.028012,-0.033951,-0.117134,-0.089555,0.038780,-0.013571
4,NSD V2,3.633467,0.315737,-0.028409,0.243366,-0.175240,-0.238593,0.030988,0.040260,-0.047659
...,...,...,...,...,...,...,...,...,...,...
283,NSD V1,2.083249,0.152464,0.110162,0.118463,-0.027632,0.002130,-0.051708,0.047027,-0.014126
284,NSD V2,3.077891,0.115714,0.530628,0.206068,-0.124446,0.130565,-0.132573,0.054470,-0.039374
285,NSD V3,4.865428,0.188001,0.155039,-0.040581,0.007743,-0.947664,0.257600,0.052707,-0.002046
286,NSD V1,2.276826,0.150028,0.164569,0.068514,-0.011693,-0.065665,-0.046767,0.061233,-0.020178


In [45]:
PARAMS_2D = ['sigma', 'slope', 'intercept', 'p_1', 'p_2', 'p_3', 'p_4', 'A_1', 'A_2']
ci_df = utils.calculate_confidence_interval(dset_df, PARAMS_2D, 'precision', n_samples=dset_df['sub'].nunique(), groupby=['dset_type'])
ci_df

,dset_type,sigma_ci_68,slope_ci_68,intercept_ci_68,p_1_ci_68,p_2_ci_68,p_3_ci_68,p_4_ci_68,A_1_ci_68,A_2_ci_68
0,NSD V1,"[2.03, 2.36]","[0.13, 0.18]","[0.1, 0.23]","[0.03, 0.14]","[-0.05, -0.0]","[-0.17, 0.07]","[-0.12, -0.04]","[0.03, 0.08]","[-0.03, -0.01]"
1,NSD V2,"[3.32, 4.33]","[0.12, 0.24]","[0.07, 0.38]","[0.08, 0.24]","[-0.16, -0.05]","[-0.67, -0.03]","[-0.07, 0.16]","[0.03, 0.06]","[-0.04, -0.01]"
2,NSD V3,"[3.94, 5.24]","[0.08, 0.22]","[0.04, 0.26]","[-0.04, 0.13]","[-0.08, 0.01]","[-0.88, -0.2]","[-0.13, 0.23]","[0.02, 0.06]","[-0.02, -0.0]"


In [84]:
mean_df =utils.calculate_weighted_mean(final_params, PARAMS_2D, 'precision', groupby=['dset_type'])
mean_df = mean_df.set_index(['dset_type']).transpose().reset_index().rename_axis(None, axis=1).rename(columns={'index': 'param'})
mean_df = np.round(mean_df, 3)
mean_df

,param,Broderick et al. V1,NSD V1,NSD V2,NSD V3
0,sigma,2.276,2.199,3.798,4.461
1,slope,0.115,0.150,0.171,0.168
2,intercept,0.372,0.171,0.254,0.202
3,p_1,0.071,0.081,0.162,0.052
4,p_2,-0.028,-0.022,-0.103,-0.047
5,p_3,0.058,-0.037,-0.342,-0.464
6,p_4,-0.005,-0.066,0.039,0.042
7,A_1,0.045,0.058,0.046,0.044
8,A_2,-0.005,-0.019,-0.025,-0.012


In [97]:
# Combine ci_df and mean_df so each cell has mean and CI list, e.g. '2.276, [2.103, 2.44]'
combined = mean_df.copy()
for col in combined.columns[1:]:  # skip 'param'
    for i, param in enumerate(combined['param']):
        # Get mean and CI for this param and dset_type
        mean_val = combined.loc[i, col]
        # ci_df_fmt may have lists as strings, ensure it's a list
        ci_val = ci_df_fmt.loc[i, col]
        # If ci_val is a numpy array, convert to list
        if hasattr(ci_val, 'tolist'):
            ci_val = ci_val.tolist()
        # Format as requested: 'mean, [low, high]'
        combined.loc[i, col] = f"{mean_val}, {ci_val}"

combined



,param,Broderick et al. V1,NSD V1,NSD V2,NSD V3
0,sigma,"2.276, [2.103, 2.44]","2.199, [2.062, 2.343]","3.798, [3.327, 4.3]","4.461, [4.083, 4.946]"
1,slope,"0.115, [0.095, 0.131]","0.15, [0.132, 0.177]","0.171, [0.133, 0.225]","0.168, [0.124, 0.213]"
2,intercept,"0.372, [0.304, 0.434]","0.171, [0.109, 0.205]","0.254, [0.121, 0.356]","0.202, [0.094, 0.284]"
3,p_1,"0.071, [0.042, 0.096]","0.081, [0.041, 0.131]","0.162, [0.102, 0.221]","0.052, [-0.01, 0.111]"
4,p_2,"-0.028, [-0.04, -0.023]","-0.022, [-0.042, -0.011]","-0.103, [-0.149, -0.068]","-0.047, [-0.08, 0.004]"
5,p_3,"0.058, [0.035, 0.091]","-0.037, [-0.099, 0.043]","-0.342, [-0.606, -0.06]","-0.464, [-0.876, -0.124]"
6,p_4,"-0.005, [-0.017, 0.009]","-0.066, [-0.104, -0.037]","0.039, [-0.057, 0.15]","0.042, [-0.099, 0.196]"
7,A_1,"0.045, [0.034, 0.054]","0.058, [0.04, 0.071]","0.046, [0.032, 0.058]","0.044, [0.027, 0.058]"
8,A_2,"-0.005, [-0.011, 0.002]","-0.019, [-0.025, -0.014]","-0.025, [-0.039, -0.012]","-0.012, [-0.019, -0.005]"
